In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Exploring Datasets

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%%time
train = pd.read_csv('/content/drive/MyDrive/Amazon Datasets/train.csv',escapechar = "\\" ,quoting = 3)

KeyboardInterrupt: ignored

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Amazon Datasets/test.csv',escapechar = "\\" ,quoting = 3)

In [ ]:
train.head()

In [ ]:
train.BROWSE_NODE_ID.value_counts()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train[train['TITLE'].isnull() &train['DESCRIPTION'].isnull()& train['BULLET_POINTS'].isnull()].shape

In [ ]:
print(train.shape)
train = train[~(train['TITLE'].isnull() &train['DESCRIPTION'].isnull()& train['BULLET_POINTS'].isnull())]
print(train.shape)

In [ ]:
train.head(10)

In [ ]:
train[train['BULLET_POINTS'].isnull()]

In [ ]:
train['DESCRIPTION'][6]

In [ ]:
train['TITLE'][8]

In [ ]:
train['BULLET_POINTS'][8]

# Text summarizer

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

In [ ]:
nltk.download('stopwords')   
nltk.download('punkt')
nltk.download('wordnet')
!pip install -q wordcloud
import wordcloud

In [ ]:
text = train['DESCRIPTION'][6]

In [ ]:
##Tokenizing the text
stopWords=stopwords.words('english')
words=word_tokenize(text)

In [ ]:
##creating freq table for each word
freqTable = dict()
for word in words:
  word=word.lower()
  if word in stopWords:
    continue
  if word in freqTable:
    freqTable[word]+=1
  else:
    freqTable[word]=1  
##creating dict to keep score of each sentence
sentenceValue = dict()
sentences = sent_tokenize(text)
for sentence in sentences:
  for word,freq in freqTable.items():
    if word in sentence.lower():
      if sentence in sentenceValue:
        sentenceValue[sentence]+=freq
      else:
        sentenceValue[sentence]=freq  

In [ ]:
sentences

In [ ]:
freqTable,sentenceValue

In [ ]:
sumvalues=0
for sentence in sentenceValue:
  sumvalues+=sentenceValue[sentence]

In [ ]:
##Avg value of sentence from og text
avg = int(sumvalues/len(sentenceValue)-1)

In [ ]:
avg

In [ ]:
##storing sentence in summary
summary=''
for sentence in sentences:
  if (sentence in sentenceValue) and (sentenceValue[sentence]>(1.2*avg)):
    summary=summary +" " +sentence
    

In [ ]:
train['TITLE'][8]+','+summary

# Final Packing into Functions 


In [ ]:
def text_summarizer(text):
  ##Tokenizing the text
  stopWords=stopwords.words('english')
  words=word_tokenize(text)
  ##creating freq table for each word
  freqTable = dict()
  for word in words:
    word=word.lower()
    if word in stopWords:
      continue
    if word in freqTable:
      freqTable[word]+=1
    else:
      freqTable[word]=1  
  ##creating dict to keep score of each sentence
  sentenceValue = dict()
  sentences = sent_tokenize(text)
  if len(sentences)==1:
    return sentences[0]
  else:  
    for sentence in sentences:
      for word,freq in freqTable.items():
        if word in sentence.lower():
          if sentence in sentenceValue:
            sentenceValue[sentence]+=freq
          else:
            sentenceValue[sentence]=freq 

    sumvalues=0
    for sentence in sentenceValue:
      sumvalues+=sentenceValue[sentence]
    ##Avg value of sentence from og text
    avg = int(sumvalues/len(sentenceValue))  
    ##storing sentence in summary
    summary=''
    for sentence in sentences:
      if (sentence in sentenceValue) and (sentenceValue[sentence]>(1.2*avg)):
        summary=summary +" " +sentence        
    return summary


In [ ]:
def bullet_points_generator(data):
  data1= data[data['BULLET_POINTS'].isnull()]
  data1_index = list(data1.index)
  for index in data1_index:
    if (data1['DESCRIPTION'].isnull()[index]==False) and (data1['TITLE'].isnull()[index]==False) :#if none are null value
      data1.at[index,'BULLET_POINTS']=[data1['TITLE'].loc[index]+','+ text_summarizer(data1['DESCRIPTION'].loc[index])]
    elif (data1['DESCRIPTION'].isnull()[index]==False) and (data1['TITLE'].isnull()[index]==True):#if TITLE is null
      data1.at[index,'BULLET_POINTS']=[text_summarizer(data1['DESCRIPTION'].loc[index])]
    elif (data1['DESCRIPTION'].isnull()[index]==True) and (data1['TITLE'].isnull()[index]==False):#if DESCRIPTION is null
      data1.at[index,'BULLET_POINTS']=  [data1['TITLE'].loc[index]]
    else:
      continue  
      ##assuming that in test data all three won't be null at the same time and removed such rows from train data
  data.drop(data1_index,axis=0,inplace=True)
  final_data = pd.concat([data,data1])
  final_data = final_data.reset_index().iloc[:,1:]
  return final_data      
      





In [ ]:
train

### Getting Final Dataaset - Beware of the below cell, as this took 2:30 hr to execute 

In [ ]:
#train_final = bullet_points_generator(train) 


In [ ]:
#train_final

In [ ]:
#train_final.to_csv('final_train.csv',index=False)

In [ ]:
#test_final = bullet_points_generator(test)

In [ ]:
#test_final.to_csv('final_test.csv',index=False)

In [ ]:
#test_final

In [ ]:
test.isnull().sum()

In [ ]:
#test_final.isnull().sum()

# Upto POS tagger pipeline

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk import word_tokenize, pos_tag, pos_tag_sents
from sklearn.pipeline import Pipeline

pd.options.display.max_colwidth = 10000000
pd.options.mode.chained_assignment = None  

In [ ]:
from string import punctuation
Sp_C = list(punctuation)

ord = ['FW','NN','NNP','NNS',"JJ",'JJr','JJS']

In [ ]:
train.drop(['TITLE', 'DESCRIPTION','BRAND'], axis=1,inplace=True)
train.dropna(inplace=True)

In [ ]:
class DataframeFunctionTransformer():
    def __init__(self, func):
        self.func = func

    def transform(self, input_df, **transform_params):
        return self.func(input_df)

    def fit(self, X, y=None, **fit_params):
        return self

In [ ]:
VSize = []


In [ ]:
def upeer_dataframe(input_df):
    input_df["BULLET_POINTS"] = input_df["BULLET_POINTS"].map(lambda t: t.lower())
    return input_df

def Stopword_dataframe(input_df):
    input_df["BULLET_POINTS"] = input_df["BULLET_POINTS"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    return input_df

def Clean(input_df):
  input_df["BULLET_POINTS"]  = input_df["BULLET_POINTS"].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
  input_df["BULLET_POINTS"] = input_df["BULLET_POINTS"].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
  return input_df

def PosTag_dataframe(input_df):
  for row in input_df.iterrows():
    print(row[0])
    # print(row[1]['BULLET_POINTS'])
    texts = row[1]['BULLET_POINTS'].split(',')
    # print(texts)
    tagged_texts = pos_tag_sents(map(word_tokenize, texts))
    req = []
    for i in tagged_texts:
      for j in i:
        if(j[0] not in Sp_C and j[1] in ord):
          if(j[0] not in req):
            req.append(j[0])
          if(j[0] not in VSize):
            VSize.append(j[0])
    sent = ""
    Max_Len = 512;
    for i in range(len(req)):
      if(i<len(req)-1 and req[i].isdigit() and req[i+1].isdigit()):
        sent += req[i] + "."
      else:
        sent += req[i] +' '
      if(i>Max_Len):
        break;
    input_df['BULLET_POINTS'].loc[row[0]]= sent
  return input_df

In [ ]:
# The pipeline 
pipeline = Pipeline([
    ("lowercase", DataframeFunctionTransformer(upeer_dataframe)),
    ("Stopword",DataframeFunctionTransformer(Stopword_dataframe)),
    ("Clean",DataframeFunctionTransformer(Clean)),
    ("Pos_Tag",DataframeFunctionTransformer(PosTag_dataframe))
])


In [ ]:

# apply the pipeline to the input dataframe
pipeline.fit_transform(train.loc[0:10000])

# Models

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
Courpus = list(train['BULLET_POINTS'].loc[0:10000])
Vcab = len(VSize)
Vcab

In [ ]:
train[0:23387]

In [ ]:
Y = list(train['BROWSE_NODE_ID'].loc[0:10000])
print("MAX = " ,max(Y))
print("MIN = " ,min(Y))

In [ ]:
One_Hot = [one_hot(words,Vcab)for words in Courpus] 

In [ ]:
One_Hot

In [ ]:
sent_length = 512
Padded = pad_sequences(One_Hot,padding='pre',maxlen=sent_length)

In [ ]:
import numpy as np
X_final=np.array(Padded)
y_final=np.array(Y)
X_final.shape,y_final.shape

In [ ]:
from sklearn.svm import SVC,SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,ExtraTreesRegressor,ExtraTreesClassifier,VotingRegressor,VotingClassifier,AdaBoostRegressor,AdaBoostClassifier
from xgboost import XGBClassifier,XGBRegressor

In [ ]:
std_scaler = StandardScaler()
##model1_pipeline = Pipeline([('scaler',std_scaler),('svc',SVC())])
svr=SVR()
svc=SVC()

# decision = DecisionTreeRegressor()
# random_forest = RandomForestRegressor(n_estimators=200)
# extra_trees = ExtraTreesRegressor(n_estimators=200)
# adaboost = AdaBoostRegressor(n_estimators=100)
# xg= XGBRegressor()

decision = DecisionTreeClassifier()
random_forest = RandomForestClassifier(n_estimators=200)
extra_trees = ExtraTreesClassifier(n_estimators=200)
adaboost = AdaBoostClassifier(n_estimators=100)
xg= XGBClassifier()

In [ ]:
#test_final.shape

In [ ]:
X_final

In [ ]:
train

# validation set

In [ ]:
X_vali,y_vali= train.loc[10001:11000],train['BROWSE_NODE_ID'].loc[10001:11000]

In [ ]:
len(y_vali)

In [ ]:
y_vali = list(y_vali)

In [ ]:
pipeline.fit_transform(X_vali)

In [ ]:
Courpus = list(train['BULLET_POINTS'].loc[10001:11000])
Vcab = len(VSize)
One_Hot = [one_hot(words,Vcab)for words in Courpus] 
sent_length = 512
Padded = pad_sequences(One_Hot,padding='pre',maxlen=sent_length)
X_vali_final=np.array(Padded)
X_vali_final.shape

In [ ]:
#predictions = model1_pipeline.predict(X_vali_final)

In [ ]:
#len(predictions),len(y_vali)

In [ ]:
#min(y_vali),max(y_vali)

In [ ]:
#m#in(predictions),max(predictions)

In [ ]:
#new_test = test_final.drop(['PRODUCT_ID','TITLE','DESCRIPTION','BRAND'],axis=1)

In [ ]:
#Ptest = new_test.loc[0:10000]

In [ ]:
#pipeline.fit_transform(Ptest)

In [ ]:
# Courpus = list(Ptest['BULLET_POINTS'])
# One_Hot = [one_hot(words,Vcab)for words in Courpus] 
# sent_length = 512
# Padded = pad_sequences(One_Hot,padding='pre',maxlen=sent_length)
# test_pred = np.array(Padded)

In [ ]:
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score
for model in [extra_trees,adaboost,xg]:
  model.fit(X_final,y_final)
  prediction = model.predict(X_vali_final)
  prediction=np.ceil(prediction)
  
  # if model==svc:
  #   print('Model= ','svc')
  #   print('accuracy= ', accuracy_score(y_vali,prediction))
  # elif model==decision:
  #   print('Model= ','Decision Tree Regressor')
  #   print('accuracy= ', accuracy_score(y_vali,prediction))
  # elif model==random_forest:
  #   print('Model= ','Random Forest Regressor')
  #   print('accuracy= ', accuracy_score(y_vali,prediction))
  if model==extra_trees:
    print('Model= ','Extra Tree Regressor')
    print('accuracy= ', accuracy_score(y_vali,prediction)) 
  elif model==adaboost:
    print('Model= ','Adaboost Regressor')
    print('accuracy= ', accuracy_score(y_vali,prediction))   
  else:
    print('Model= ','XG boost Regressor')
    print('accuracy= ', accuracy_score(y_vali,prediction))  


In [ ]:
#predictions = np.floor(predictions)

# Models

In [ ]:
import pandas as pd
import numpy as np  

In [ ]:
df= pd.read_csv('cluster18.csv')

In [ ]:
df.head()

,id,PRODUCT_ID,BULLET_POINTS
0,488,513,material premium class silicone cover compatib...
1,9116,9704,ferrero rocher chocolate message card bouquet ...
2,12435,13257,designer case samsung galaxy on7 prime/galaxy ...
3,13185,14044,samsung galaxy j7 prime octa-core processor ra...
4,14911,15879,designer case samsung galaxy on7 prime/galaxy ...


In [ ]:
for row in df.iterrows():
  print(row[1][2])
  print('*********************************')

material premium class silicone cover compatible device samsung galaxy j7 max color multi-coloured silicon maximum protection phone easy installation removal 
*********************************
ferrero rocher chocolate message card bouquet teddy bear cellophane wrap laxmi 
*********************************
designer case samsung galaxy on7 prime/galaxy nxt/galaxy j7 prime sm-g610f/galaxy definitive fit maximum protection slim protective trendy impressive shockproof ultra light strong polymer hard protect shocks thin device everyday tear impact resistant one-piece rear shell pocket bag instant phone access 
*********************************
samsung galaxy j7 prime octa-core processor ram non removable battery phone internal storage microsd card 
*********************************
designer case samsung galaxy on7 prime/galaxy nxt/galaxy j7 prime sm-g610f/galaxy definitive fit maximum protection slim protective trendy impressive shockproof ultra light strong polymer hard protect shocks thin 

## TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
sentences = []
for i in range(df.shape[0]):
  sentences.append(df.iloc[i]['BULLET_POINTS'])


In [ ]:
sentences

['material premium class silicone cover compatible device samsung galaxy j7 max color multi-coloured silicon maximum protection phone easy installation removal ',
 'ferrero rocher chocolate message card bouquet teddy bear cellophane wrap laxmi ',
 'designer case samsung galaxy on7 prime/galaxy nxt/galaxy j7 prime sm-g610f/galaxy definitive fit maximum protection slim protective trendy impressive shockproof ultra light strong polymer hard protect shocks thin device everyday tear impact resistant one-piece rear shell pocket bag instant phone access ',
 'samsung galaxy j7 prime octa-core processor ram non removable battery phone internal storage microsd card ',
 'designer case samsung galaxy on7 prime/galaxy nxt/galaxy j7 prime sm-g610f/galaxy definitive fit maximum protection slim protective trendy impressive shockproof ultra light strong polymer hard protect shocks thin device everyday tear impact resistant one-piece rear shell pocket bag instant phone access ',
 'product teddy bear col

In [ ]:
vectorizer = TfidfVectorizer()
sentences_vectorized = vectorizer.fit_transform(sentences)

In [ ]:
sentences_vectorized.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.32132918, 0.        ,
        0.        ],
       [0.        , 0.1601729 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.28468092,
        0.        ],
       [0.        , 0.        , 0.30991914, ..., 0.        , 0.        ,
        0.        ]])

## Word2vec

In [ ]:
from gensim.models import word2vec

In [ ]:
model = word2vec.Word2Vec(sentences)

In [ ]:
model.wv.vocab.keys()

dict_keys(['m', 'a', 't', 'e', 'r', 'i', 'l', ' ', 'p', 'u', 'c', 's', 'o', 'n', 'v', 'b', 'd', 'g', 'x', 'y', 'j', '7', '-', 'h', 'f', 'w', '/', 'k'])

In [ ]:
for i, sentence in enumerate(sentences):
  tokenized=[]
  for word in sentence.split(' '):
    if word=='':
      continue
    tokenized.append(word)
  sentences[i] = tokenized

In [ ]:
sentences

[['material',
  'premium',
  'class',
  'silicone',
  'cover',
  'compatible',
  'device',
  'samsung',
  'galaxy',
  'j7',
  'max',
  'color',
  'multi-coloured',
  'silicon',
  'maximum',
  'protection',
  'phone',
  'easy',
  'installation',
  'removal'],
 ['ferrero',
  'rocher',
  'chocolate',
  'message',
  'card',
  'bouquet',
  'teddy',
  'bear',
  'cellophane',
  'wrap',
  'laxmi'],
 ['designer',
  'case',
  'samsung',
  'galaxy',
  'on7',
  'prime/galaxy',
  'nxt/galaxy',
  'j7',
  'prime',
  'sm-g610f/galaxy',
  'definitive',
  'fit',
  'maximum',
  'protection',
  'slim',
  'protective',
  'trendy',
  'impressive',
  'shockproof',
  'ultra',
  'light',
  'strong',
  'polymer',
  'hard',
  'protect',
  'shocks',
  'thin',
  'device',
  'everyday',
  'tear',
  'impact',
  'resistant',
  'one-piece',
  'rear',
  'shell',
  'pocket',
  'bag',
  'instant',
  'phone',
  'access'],
 ['samsung',
  'galaxy',
  'j7',
  'prime',
  'octa-core',
  'processor',
  'ram',
  'non',
  'remova

In [ ]:
model = word2vec.Word2Vec(sentences,window=10,max_vocab_size=100,iter=100)

In [ ]:
model.wv.vocab

{'case': <gensim.models.keyedvectors.Vocab at 0x7fc57587a390>,
 'compatible': <gensim.models.keyedvectors.Vocab at 0x7fc57587a190>,
 'cover': <gensim.models.keyedvectors.Vocab at 0x7fc57587a150>,
 'galaxy': <gensim.models.keyedvectors.Vocab at 0x7fc57587a210>,
 'j7': <gensim.models.keyedvectors.Vocab at 0x7fc57587a290>,
 'phone': <gensim.models.keyedvectors.Vocab at 0x7fc57587a350>,
 'prime': <gensim.models.keyedvectors.Vocab at 0x7fc57587a250>,
 'protection': <gensim.models.keyedvectors.Vocab at 0x7fc57587a310>,
 'samsung': <gensim.models.keyedvectors.Vocab at 0x7fc57587a1d0>}

In [ ]:
model.wv['j7']

array([-2.9282363e-03,  4.8921299e-03,  2.8023999e-03,  2.8085483e-03,
       -1.1333256e-03, -3.1712966e-04,  6.4605125e-04,  2.5961702e-03,
        5.9721316e-03, -6.0241125e-03,  1.6388861e-03, -3.3461144e-03,
       -4.0851063e-03,  7.8770617e-04, -2.7086008e-03,  5.8149043e-03,
        5.5384496e-03,  1.4791493e-03,  8.2827751e-03,  1.0690368e-03,
       -2.5217389e-03, -4.0897592e-03,  1.3146764e-03, -1.4555649e-03,
        4.8040347e-03,  3.5720693e-03,  2.4325859e-03,  6.0524401e-04,
       -7.5191464e-03, -3.4525441e-03, -1.6709493e-03, -2.0385229e-03,
        7.4362350e-03,  4.3365811e-03,  1.3723829e-03, -4.4232067e-03,
       -4.1790958e-03, -3.9356081e-03, -1.5486235e-03, -4.7299070e-03,
       -2.9524500e-04,  5.3217966e-05,  9.8335824e-04,  4.4010086e-03,
       -2.1115034e-03,  3.6514506e-03,  2.9426867e-03, -4.1196332e-03,
        3.2707979e-03, -2.9679500e-03,  2.9316579e-03,  5.1696612e-03,
       -3.9984998e-03, -2.4369911e-03,  5.9382962e-03, -4.5033973e-03,
      

## DOC2VEC

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(sentences)]
tagged_data

[TaggedDocument(words=['material', 'premium', 'class', 'silicone', 'cover', 'compatible', 'device', 'samsung', 'galaxy', 'j7', 'max', 'color', 'multi-coloured', 'silicon', 'maximum', 'protection', 'phone', 'easy', 'installation', 'removal'], tags=[0]),
 TaggedDocument(words=['ferrero', 'rocher', 'chocolate', 'message', 'card', 'bouquet', 'teddy', 'bear', 'cellophane', 'wrap', 'laxmi'], tags=[1]),
 TaggedDocument(words=['designer', 'case', 'samsung', 'galaxy', 'on7', 'prime/galaxy', 'nxt/galaxy', 'j7', 'prime', 'sm-g610f/galaxy', 'definitive', 'fit', 'maximum', 'protection', 'slim', 'protective', 'trendy', 'impressive', 'shockproof', 'ultra', 'light', 'strong', 'polymer', 'hard', 'protect', 'shocks', 'thin', 'device', 'everyday', 'tear', 'impact', 'resistant', 'one-piece', 'rear', 'shell', 'pocket', 'bag', 'instant', 'phone', 'access'], tags=[2]),
 TaggedDocument(words=['samsung', 'galaxy', 'j7', 'prime', 'octa-core', 'processor', 'ram', 'non', 'removable', 'battery', 'phone', 'internal

In [ ]:
model = Doc2Vec(tagged_data, vector_size = 20, window = 2, min_count = 1,, epochs = 100)

'''
vector_size = Dimensionality of the feature vectors.
window = The maximum distance between the current and predicted word within a sentence.
min_count = Ignores all words with total frequency lower than this.
alpha = The initial learning rate.
'''

## Print model vocabulary
model.wv.vocab

{"'adtronics": <gensim.models.keyedvectors.Vocab at 0x7fc57587a650>,
 'absorbs': <gensim.models.keyedvectors.Vocab at 0x7fc5778aa7d0>,
 'access': <gensim.models.keyedvectors.Vocab at 0x7fc5778aaad0>,
 'adapter/wall': <gensim.models.keyedvectors.Vocab at 0x7fc57587aa90>,
 'amazing': <gensim.models.keyedvectors.Vocab at 0x7fc57585e610>,
 'android': <gensim.models.keyedvectors.Vocab at 0x7fc57587a750>,
 'availability': <gensim.models.keyedvectors.Vocab at 0x7fc5778aaa10>,
 'bag': <gensim.models.keyedvectors.Vocab at 0x7fc5778aa390>,
 'battery': <gensim.models.keyedvectors.Vocab at 0x7fc5778aaf90>,
 'bear': <gensim.models.keyedvectors.Vocab at 0x7fc575874090>,
 'body': <gensim.models.keyedvectors.Vocab at 0x7fc57585ec10>,
 'bouquet': <gensim.models.keyedvectors.Vocab at 0x7fc575874750>,
 'bow': <gensim.models.keyedvectors.Vocab at 0x7fc57585e510>,
 'brother': <gensim.models.keyedvectors.Vocab at 0x7fc5778aa890>,
 'bumps': <gensim.models.keyedvectors.Vocab at 0x7fc5778aa510>,
 'c9': <gensim

In [ ]:
len(model.wv.vocab)

169

In [44]:
list(model.docvecs[12])

array([-0.0137639 ,  0.00822021, -0.64427865,  0.66964585, -0.4617727 ,
        0.19720778,  0.68632954, -0.32569808, -0.58936596,  1.05935   ,
        0.28541023,  0.02544106, -0.21017215,  0.02848674, -0.5766824 ,
       -0.1312639 , -0.23680934,  0.27173018, -0.393212  , -0.20725958,
       -0.724784  , -0.16572446,  0.49136472,  0.09901375, -0.22169138,
       -0.72473687,  0.8304161 , -0.6650895 , -0.62676144,  0.9494103 ],
      dtype=float32)

In [45]:
list(model.docvecs[12])

[-0.013763897,
 0.008220211,
 -0.64427865,
 0.66964585,
 -0.4617727,
 0.19720778,
 0.68632954,
 -0.32569808,
 -0.58936596,
 1.05935,
 0.28541023,
 0.02544106,
 -0.21017215,
 0.028486744,
 -0.5766824,
 -0.1312639,
 -0.23680934,
 0.27173018,
 -0.393212,
 -0.20725958,
 -0.724784,
 -0.16572446,
 0.49136472,
 0.09901375,
 -0.22169138,
 -0.72473687,
 0.8304161,
 -0.6650895,
 -0.62676144,
 0.9494103]

# Applying doc2vec on train dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
def reading_data(name):
  df = pd.read_csv(name)
  return df

In [3]:
train1 = reading_data('/content/drive/MyDrive/Amazon Challenge/Train_Clusters/1.csv')
test1 = reading_data('/content/drive/MyDrive/Amazon Challenge/Test_Clusters/1.csv')

In [4]:
train1 = train1.iloc[:,2:]
test1 = test1.iloc[:,1:]

In [5]:
train1.shape,test1.shape

((44032, 2), (184, 2))

In [6]:
train1.BROWSE_NODE_ID.value_counts()

40      6115
1017    3113
1592    1987
773     1699
5       1636
        ... 
107        1
11         1
3787       1
826        1
0          1
Name: BROWSE_NODE_ID, Length: 611, dtype: int64

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
train1.head()

,BULLET_POINTS,BROWSE_NODE_ID
0,100-percent cotton 2-piece rug set super soft ...,383
1,polyester interlock climacool machine wash tha...,1481
2,polyester elastane wash colors iron regular fi...,5
3,please size chart purchase cotton elastane mac...,1171
4,cotton polyester machine wash soft cup daisy d...,773


In [12]:
test1.head()

,PRODUCT_ID,BULLET_POINTS
0,93,cotton dress height=14 chest=10.5 gentle hand ...
1,479,fit type free high quality quotes material pol...
2,1248,ampoule machine
3,1378,color off-white fabric cotton flax package con...
4,2288,color name seagreen cotton elastane machine wa...


In [7]:
def df_tokenizer(df):
  sentences = []
  for i in range(df.shape[0]):
    sentences.append(df.iloc[i]['BULLET_POINTS'])
  for i, sentence in enumerate(sentences):
    tokenized=[]
    for word in sentence.split(' '):
      if word=='':
        continue
      tokenized.append(word)
    sentences[i] = tokenized 
  return sentences     


In [8]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [9]:
Doc2Vec()

In [10]:
def df_doc2vec(df,s):
  tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(s)]
  model = Doc2Vec(tagged_data, vector_size = 100, window = 2, min_count = 1, epochs = 10)

  '''
  vector_size = Dimensionality of the feature vectors.
  window = The maximum distance between the current and predicted word within a sentence.
  min_count = Ignores all words with total frequency lower than this.
  alpha = The initial learning rate.
  '''
  word_embedding_size= model.wv.vector_size
  vector_list = []
  for i in range(df.shape[0]):
    vector_list.append(list(model.docvecs[i]))
  print('The word embedding vector size is {}'.format(word_embedding_size)) 
  return vector_list 




In [11]:
sentence_train1= df_tokenizer(train1)

In [12]:
final_vector_train1 = df_doc2vec(train1,sentence_train1)

The word embedding vector size is 100


In [13]:
y_train = np.array(train1.BROWSE_NODE_ID)

In [14]:
sentence_test1 = df_tokenizer(test1)
final_vector_test1 = df_doc2vec(test1,sentence_test1)  ##We want to find prediction for these test vectors

The word embedding vector size is 100


In [15]:
from sklearn.decomposition import PCA

In [16]:
final_vector_train1=np.array(final_vector_train1)
final_vector_test1=np.array(final_vector_test1)

In [17]:
pca = PCA(n_components=2)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(final_vector_train1,y_train,test_size=0.25)

In [20]:
from sklearn.svm import SVC,SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,ExtraTreesRegressor,ExtraTreesClassifier,VotingRegressor,VotingClassifier,AdaBoostRegressor,AdaBoostClassifier
from xgboost import XGBClassifier,XGBRegressor
from sklearn.metrics import mean_squared_error,r2_score,accuracy_score

In [21]:
def Fitting_models_with_evaluation(x_train,y_train,x_test):
  model = LogisticRegression(n_jobs=-1,max_iter=1)
  model.fit(x_train,y_train)
  predictions = model.predict(x_test)
  return predictions

In [76]:
test_finally =pd.DataFrame({'PRODUCT_ID':[],'BROWSE_NODE_ID':[]})

In [93]:
for i in range(50):
  train = reading_data('/content/drive/MyDrive/Amazon Challenge/Train_Clusters/{}.csv'.format(i))
  test = reading_data('/content/drive/MyDrive/Amazon Challenge/Test_Clusters/{}.csv'.format(i))
  train = train.iloc[:,2:]
  test = test.iloc[:,1:]
  product_id = list(test['PRODUCT_ID'])
  sentence_train = df_tokenizer(train)
  final_vector_train = df_doc2vec(train,sentence_train)
  y_train = np.array(train.BROWSE_NODE_ID)

  sentence_test = df_tokenizer(test)
  final_vector_test = df_doc2vec(test,sentence_test1) 
  final_vector_train =np.array(final_vector_train)
  final_vector_test =np.array(final_vector_test) 
  prediction = Fitting_models_with_evaluation(final_vector_train,y_train,final_vector_test)
  df=pd.DataFrame({'PRODUCT_ID':product_id,'BROWSE_NODE_ID':prediction})
  test_finally.append(df)


The word embedding vector size is 20


KeyError: ignored

In [63]:
X_train.shape,y_train.shape

((33024, 20), (33024,))

In [64]:
#temp = pca.fit_transform(X_train)

In [65]:
#temp.shape

(33024, 2)

In [ ]:
svr=SVR()
svr.fit(X=temp,y= y_train)
pred = svr.predict(X_test)
print(mean_squared_error(y_test,pred))

In [29]:
decision =DecisionTreeRegressor()
decision.fit(temp,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [30]:
pred = decision.predict(pca.fit_transform(X_test))

In [31]:
print(mean_squared_error(y_test,np.ceil(pred)))

891408290.9202398


In [ ]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)
pred=rf.predict(X_test)

In [ ]:
accuracy_score(y_test,pred)

In [74]:
logit = LogisticRegression(n_jobs=-1,max_iter=5)
logit.fit(X_train,y_train)
pred=logit.predict(X_test)
accuracy_score(y_test,pred)

0.13971656976744187

In [32]:
# logit = SGDClassifier()
# logit.fit(temp,y_train)
# pred=logit.predict(pca.fit_transform(X_test))
# accuracy_score(y_test,pred)

0.07258357558139535

In [75]:
logit.predict(final_vector_test1)

array([40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40])

In [34]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Flatten
# from keras.layers.embeddings import Embedding
# # define problem
# vocab_size = 100
# max_length = 2
# # define the model
# model = Sequential()
# model.add(Embedding(vocab_size, 8, input_length=max_length))
# model.add(Flatten())
# model.add(Dense(1, activation='sigmoid'))
# # compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# # summarize the model
# print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 8)              800       
_________________________________________________________________
flatten_1 (Flatten)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 817
Trainable params: 817
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
model.fit(temp,y_train)

1032/1032 [==============================] - 9s 1ms/step - loss: -6380.2772 - accuracy: 0.0000e+00


In [36]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 8)              800       
_________________________________________________________________
flatten_1 (Flatten)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 817
Trainable params: 817
Non-trainable params: 0
_________________________________________________________________
None
